In [1]:
from langchain_groq import ChatGroq

In [90]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-8b-instant"
)


In [91]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.ibm.com/job/20978143/software-developer-intern-2025-san-jose-ca/?codes=WEB_Search_INDIA ")
page_data = loader.load().pop().page_content
print(page_data)




















Software Developer Intern: 2025 - US | IBM
















































 



Skip to content



Toggle NavigationIBM CareersEnglish简体中文FrançaisFrançais Canadien日本語PortuguêsEspañolDeutschIBM Careers IBM Careers      English    简体中文    Français    Français Canadien    日本語    Português    Español    Deutsch      
 





Job Details2024-01-04T09:25:13+00:00 

724766BR


Software Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now Software Developer Intern: 2025    Multiple Locations See All  San Jose  Lowell  Poughkeepsie  Austin    Software Engineering Internship   Apply now IntroductionAt IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something better, but to attempt things you’ve never thought possible. Are you ready to lead i

In [92]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [93]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Developer Intern: 2025',
 'experience': 'Summer 2025',
 'skills': ['Programming Languages: Familiarity with one or more programming languages such as Java, Python, C++, or JavaScript.',
  'Development Tools: Experience with software development tools such as Git, IDEs (e.g., Eclipse, IntelliJ), and version control systems.',
  'Problem-Solving Skills: Strong analytical and problem-solving abilities with a keen attention to detail.',
  'Communication: Excellent verbal and written communication skills, with the ability to collaborate effectively in a team environment.',
  'Adaptability: Willingness to learn new technologies and adapt to changing project requirements.',
  'Teamwork: Ability to work both independently and as part of a collaborative team.'],
 'description': 'At IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something b

In [94]:
type(json_res)

dict

In [95]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [96]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [97]:
job = json_res
job['skills']

links = collection.query(query_texts=job['skills'], n_results=1).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'}]]

In [98]:
job

{'role': 'Software Developer Intern: 2025',
 'experience': 'Summer 2025',
 'skills': ['Programming Languages: Familiarity with one or more programming languages such as Java, Python, C++, or JavaScript.',
  'Development Tools: Experience with software development tools such as Git, IDEs (e.g., Eclipse, IntelliJ), and version control systems.',
  'Problem-Solving Skills: Strong analytical and problem-solving abilities with a keen attention to detail.',
  'Communication: Excellent verbal and written communication skills, with the ability to collaborate effectively in a team environment.',
  'Adaptability: Willingness to learn new technologies and adapt to changing project requirements.',
  'Teamwork: Ability to work both independently and as part of a collaborative team.'],
 'description': 'At IBM, work is more than a job – it’s a calling: To build. To design. To code. To consult. To think along with clients and sell. To make markets. To invent. To collaborate. Not just to do something b

In [99]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sanya, a business development executive at Juhu Industries. Juhu Industries is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Juhu Industries
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Juhu Industries's portfolio: {link_list}
        Remember you are Sanya, BDE at Juhu Industries. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Business Potential with Juhu Industries

Dear [Recipient's Name],

I came across your job posting for a Software Developer Intern at IBM and was impressed by the exciting opportunities you're offering. As a business development executive at Juhu Industries, I'd like to introduce you to our AI & Software Consulting company that specializes in facilitating seamless business process integration through automated tools.

At Juhu Industries, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise spans a wide range of technologies, including machine learning, DevOps, and software development.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* **Machine Learning with Python**: Check out our work on [https://example.com/ml-python-portfolio](https://example.com/ml-python-portfolio), where we developed a pre